In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install plotly

In [11]:
import plotly.graph_objects as go
import numpy as np
from scipy.interpolate import griddata
import plotly.graph_objects as go
import numpy as np
import pandas as pd

In [4]:

#Cargar el archivo .csv

file_path = '/content/drive/MyDrive/Student_Performance.csv'

df = pd.read_csv(file_path)

,Hours Studied,Previous Scores,Extracurricular Activities,Sleep Hours,Sample Question Papers Practiced,Performance Index
0,7,99,Yes,9,1,91.0
1,4,82,No,4,2,65.0
2,8,51,Yes,7,2,45.0
3,5,52,Yes,5,2,36.0
4,7,75,No,8,5,66.0
...,...,...,...,...,...,...
9995,1,49,Yes,4,2,23.0
9996,7,64,Yes,8,5,58.0
9997,6,83,Yes,8,5,74.0
9998,9,97,Yes,7,0,95.0


In [19]:
from scipy.interpolate import griddata
import plotly.graph_objects as go
import numpy as np


study_col = "Hours Studied"
sleep_col = "Sleep Hours"
perf_col  = "Performance Index"

# Definir bins
study_bins = np.arange(df[study_col].min(), df[study_col].max() + 1, 1)
sleep_bins = np.arange(df[sleep_col].min(), df[sleep_col].max() + 1, 1)

df_binned = df.copy()
df_binned["study_bin"] = pd.cut(df_binned[study_col], bins=study_bins, include_lowest=True)
df_binned["sleep_bin"] = pd.cut(df_binned[sleep_col], bins=sleep_bins, include_lowest=True)

# 3) Tabla 2D de medias de rendimiento
pivot = (
    df_binned
    .groupby(["study_bin", "sleep_bin"], observed=True)[perf_col]
    .mean()
    .unstack()
)

# Centros de los intervalos
x_centers = pivot.index.map(lambda iv: iv.mid).to_numpy()
y_centers = pivot.columns.map(lambda iv: iv.mid).to_numpy()

Z = pivot.to_numpy()


contour = go.Contour(
    x=x_centers, y=y_centers, z=Z.T,
    contours=dict(
        coloring="heatmap",
        showlabels=True, labelfont=dict(size=12)
    ),
    colorbar=dict(title="Nota media"),
    hovertemplate="Horas estudio=%{x:.1f}<br>Horas sueño=%{y:.1f}<br>Nota media=%{z:.1f}<extra></extra>"
)



fig = go.Figure(data=[contour])
fig.update_layout(
    title="RENDIMIENTO ACADEMICO SEGÚN HORAS DE ESTUDIO Y SUEÑO",
    xaxis_title="Horas de Estudio",
    yaxis_title="Horas de Sueño"
)
fig.show()